# Salient Predictions Ski-Cast

In November, Salient predicts snow accumulation at 90 IKON and Epic resorts.


In [ ]:
import datetime
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

try:
    import salientsdk as sk
except ModuleNotFoundError as e:
    if os.path.exists("../salientsdk"):
        sys.path.append(os.path.abspath(".."))
        import salientsdk as sk
    else:
        raise ModuleNotFoundError("Install salient SDK with: pip install salientsdk")

force = False

year = datetime.datetime.now().year
today = datetime.date.today().strftime("%Y-%m-%d")
start_month = 10  # first day of the ski season
end_month = 5  # final day of the ski season
fcst_date = f"{year}-10-15"
hist_start = f"1990-{start_month}-01"
fcst_end = f"{year+1}-{end_month}-01"
vars = ["temp", "precip"]


sk.set_file_destination("ski_example")
sk.login("username", "password")

In [ ]:
#
resorts = {
    "japan": pd.DataFrame(
        [
            {"lon": 137.861, "lat": 36.690, "name": "Hakuba"},  # epic
            {"lon": 140.687, "lat": 42.824, "name": "Rusutsu"},  # epic
            {"lon": 140.685, "lat": 42.824, "name": "Niseko"},  # ikon
            {"lon": 140.685, "lat": 42.824, "name": "Lotte Arai"},  # ikon
            # Tazawako indy
            # Okunakayama
        ]
    ),
    "alps": pd.DataFrame(
        [
            {"lon": 6.8632749, "lat": 45.924065, "name": "Chamonix"},  # ikon
            {"lon": 7.7522747, "lat": 46.0222204, "name": "Zermatt"},  # ikon
            {"lon": 8.5916293, "lat": 46.6324621, "name": "Andermatt-Sedrun"},  # epic
            {"lon": 12.3925407, "lat": 47.4492375, "name": "Kitzbuhel"},  # ikon
        ]
    ),
    "pnw": pd.DataFrame(
        [
            {"lon": -123.204545, "lat": 49.396018, "name": "Cypress"},  # ikon
            {"lon": -121.6781891, "lat": 44.0024701, "name": "Bachelor"},  # ikon
            {"lon": -121.0890197, "lat": 47.7448119, "name": "Stevens Pass"},  # epic
            {"lon": -122.9486474, "lat": 50.1149639, "name": "Whistler"},  # epic
            {"lon": -121.4747533, "lat": 46.9352963, "name": "Crystal Mtn"},  # ikon
            {"lon": -121.4257485, "lat": 47.4442426, "name": "Alpental"},  # ikon
            {"lon": -121.4164161, "lat": 47.4245711, "name": "Snoqualmie"},  # ikon
        ]
    ),
    "rockies": pd.DataFrame(
        [
            # The four Aspen resorts all perform similarly.  Combine into one:
            # {"lon": -106.9490961, "lat": 39.2083984, "name": "Aspen Snowmass"},
            # {"lon": -106.8610687, "lat": 39.2058029, "name": "Buttermilk"},
            # {"lon": -106.8553613, "lat": 39.1824124, "name": "Aspen Highlands"},
            {"lon": -106.818227, "lat": 39.1862601, "name": "Aspen Mtn"},  # ikon
            {"lon": -106.8045169, "lat": 40.4571991, "name": "Steamboat"},  # ikon
            # Beaver Creek and Vail perform similarly and are right next to each other
            # {"lon": -106.5167109, "lat": 39.6042863, "name": "Beaver Creek"}, # epic
            {"lon": -106.3549717, "lat": 39.6061444, "name": "Vail"},  # epic
            {"lon": -106.1516265, "lat": 39.501419, "name": "Copper"},  # ikon
            {"lon": -106.0676088, "lat": 39.4808351, "name": "Breckenridge"},  # epic
            {"lon": -105.9437656, "lat": 39.6075962, "name": "Keystone"},  # epic
            {"lon": -105.8719397, "lat": 39.6425118, "name": "A-Basin"},  # ikon
            {"lon": -105.762488, "lat": 39.8868392, "name": "Winter Park"},  # ikon
            {"lon": -105.5826786, "lat": 39.9372203, "name": "Eldora"},  # ikon
        ]
    ),
    "new_england": pd.DataFrame(
        [
            {"lon": -72.9278443, "lat": 43.0906207, "name": "Stratton"},  # ikon
            {"lon": -72.9204014, "lat": 42.9602444, "name": "Mt Snow"},  # epic
            {"lon": -72.8944139, "lat": 44.1359019, "name": "Sugarbush"},  # ikon
            # {"lon": -72.842512, "lat": 43.6621499, "name": "Pico"}, # ikon near Killington
            {"lon": -72.7967531, "lat": 43.6262922, "name": "Killington"},  # ikon
            {"lon": -72.7814124, "lat": 44.5303066, "name": "Stowe"},  # epic
            {"lon": -72.7170416, "lat": 43.4018257, "name": "Okemo"},  # epic
            {"lon": -72.08014, "lat": 43.331889, "name": "Sunapee"},  # epic
            {"lon": -71.8655176, "lat": 43.0198715, "name": "Crotched"},  # epic
            {"lon": -71.6336041, "lat": 44.0563456, "name": "Loon"},  # ikon
            {"lon": -71.2393036, "lat": 44.2640724, "name": "Wildcat"},  # epic
            {"lon": -71.229443, "lat": 44.082771, "name": "Attitash"},  # epic
            {"lon": -70.8568727, "lat": 44.4734182, "name": "Sunday River"},  # ikon
            {"lon": -70.3085109, "lat": 45.0541811, "name": "Sugarloaf"},  # ikon
        ]
    ),
    "europe": pd.DataFrame(
        [
            {"lon": 1.4707674, "lat": 42.5729217, "name": "Arinsal"},  # ikon
            {"lon": 1.499825, "lat": 42.6317345, "name": "Ordino Arcalís"},  # ikon
            {"lon": 1.6462281, "lat": 42.5783833, "name": "Grandvalira"},  # ikon
            {"lon": 11.6520936, "lat": 46.5739752, "name": "Dolomiti"},  # ikon
        ]
    ),
    "na_west": pd.DataFrame(
        [
            {"lon": -120.2483913, "lat": 39.1906091, "name": "Palisades Tahoe"},  # ikon
            {"lon": -120.1210934, "lat": 39.2745678, "name": "Northstar"},  # epic
            {"lon": -120.0651665, "lat": 38.6847514, "name": "Kirkwood"},  # epic
            {"lon": -119.9428424, "lat": 38.9569241, "name": "Heavenly"},  # epic
            {"lon": -119.8859331, "lat": 50.8844311, "name": "Sun Peaks"},  # ikon
            {"lon": -119.0906293, "lat": 37.7679169, "name": "June"},  # ikon
            {"lon": -119.0267806, "lat": 37.6510972, "name": "Mammoth"},  # ikon
            {"lon": -118.1630779, "lat": 50.9583858, "name": "Revelstoke"},  # ikon
            {"lon": -117.8194705, "lat": 49.1024147, "name": "RED"},  # ikon
            {"lon": -117.036177, "lat": 34.2248821, "name": "Snow Valley"},  # ikon
            {"lon": -116.8892717, "lat": 34.2364081, "name": "Snow Summit"},  # ikon
            {"lon": -116.8608572, "lat": 34.2276766, "name": "Bear Mtn"},  # ikon
            {"lon": -116.6227441, "lat": 48.3679757, "name": "Schweitzer"},  # ikon
            {"lon": -116.2380671, "lat": 50.4602801, "name": "Panorama"},  # ikon
            {"lon": -116.1621717, "lat": 51.4419206, "name": "Lake Louise"},  # ikon
            {"lon": -115.7840699, "lat": 51.0780997, "name": "Banff"},  # ikon
            {"lon": -115.5982699, "lat": 51.2037624, "name": "Norquay"},  # ikon
            {"lon": -115.5707632, "lat": 51.1751675, "name": "SkiBig3"},  # ikon
            {"lon": -114.3542874, "lat": 43.6949128, "name": "Sun Valley"},  # ikon
            {"lon": -114.3461537, "lat": 43.6820566, "name": "Dollar Mtn"},  # ikon
            {"lon": -111.8571529, "lat": 41.2161404, "name": "Snowbasin"},  # ikon
            # The four Cottonwood Canyon resorts all perform similarly.  Combine:
            # {"lon": -111.6563885, "lat": 40.5810814, "name": "Snowbird"},
            {"lon": -111.6385807, "lat": 40.5884218, "name": "Alta"},  # ikon
            # {"lon": -111.591885, "lat": 40.619852, "name": "Solitude"},
            # {"lon": -111.583187, "lat": 40.598019, "name": "Brighton"},
            {"lon": -111.5079947, "lat": 40.6514199, "name": "Park City"},  # epic
            {"lon": -111.478306, "lat": 40.63738, "name": "Deer Valley"},  # ikon
            {"lon": -111.4012076, "lat": 45.2857289, "name": "Big Sky"},  # ikon
            {"lon": -110.8279183, "lat": 43.5875453, "name": "Jackson Hole"},  # ikon
            {"lon": -106.9878231, "lat": 38.8697146, "name": "Crested Butte"},  # ikon
            {"lon": -105.4545, "lat": 36.5959999, "name": "Taos"},  # ikon
        ]
    ),
    "na_east": pd.DataFrame(
        [
            {"lon": -94.9707416, "lat": 39.4673048, "name": "Snow Creek"},  # epic- Kansas City
            {"lon": -92.7878062, "lat": 44.8576608, "name": "Afton"},  # epic - Minneapolis
            {"lon": -90.6506898, "lat": 38.5353168, "name": "Hidden Valley"},  # epic
            {"lon": -88.1876602, "lat": 42.4989548, "name": "Wilmot"},  # epic
            {"lon": -86.5122305, "lat": 38.5555868, "name": "Paoli"},  # epic
            {"lon": -84.930067, "lat": 45.162884, "name": "Boyne"},  # ikon
            # {"lon": -84.926535, "lat": 45.4647239, "name": "Boyne Highlands"},  # ikon
            {"lon": -83.8115217, "lat": 42.54083, "name": "Mt. Brighton"},  # epic
            {"lon": -83.6777778, "lat": 40.3180556, "name": "Mad River"},  # epic
            {"lon": -81.5632108, "lat": 41.2640987, "name": "Boston Mills"},  # epic
            {"lon": -81.259745, "lat": 41.52687, "name": "Alpine Valley"},  # epic
            {"lon": -80.3122216, "lat": 44.5037818, "name": "Blue Mtn"},  # ikon
            {"lon": -79.9960444, "lat": 38.4118566, "name": "Snowshoe"},  # ikon
            {"lon": -79.2977032, "lat": 40.0229768, "name": "7 Springs"},  # epic
            {"lon": -79.2581204, "lat": 40.058031, "name": "Hidden Valley 2"},  # epic
            {"lon": -79.1657908, "lat": 40.1638728, "name": "Laurel"},  # epic
            {"lon": -77.9333126, "lat": 39.7417652, "name": "Whitetail"},  # epic
            {"lon": -77.375459, "lat": 39.76366, "name": "Liberty"},  # epic
            {"lon": -76.9275492, "lat": 40.1094506, "name": "Roundtop"},  # epic
            {"lon": -75.6563315, "lat": 41.1091686, "name": "Jack Frost"},  # epic
            {"lon": -75.601282, "lat": 41.050189, "name": "Big Boulder"},  # epic
            {"lon": -74.5852526, "lat": 46.2096417, "name": "Tremblant"},  # ikon
            {"lon": -74.2567116, "lat": 42.2937298, "name": "Windham"},  # ikon
            {"lon": -74.2246402, "lat": 42.2028811, "name": "Hunter"},  # epic
        ]
    ),
}

# Assign a color to each region for later plotting purposes, using
# Tol's colorblind-friendly "vibrant" palette.
# https://cran.r-project.org/web/packages/khroma/vignettes/tol.html
colors = {
    "japan": "#004488",  # blue
    "alps": "#33BBEE",  # cyan
    "pnw": "#009988",  # teal
    "rockies": "#CC3311",  # red
    "new_england": "#DDAA33",  # yellow
    "europe": "#555555",  # dark grey
    "na_west": "#666666",  # grey
    "na_east": "#777777",  # light grey
}

In [ ]:
geo_files = {
    region: sk.upload_location_file(
        lats=geo["lat"],
        lons=geo["lon"],
        names=geo["name"],
        geoname=f"ski_resorts_{region}",
        force=force,
    )
    for region, geo in resorts.items()
}

# We will later use a Location object to query the Salient API
# The functions are capable of handling multiple location files,
# so we can pass a vector here.
ski_locs = sk.Location(location_file=list(geo_files.values()))
print(ski_locs)

location file: ['ski_resorts_japan.csv', 'ski_resorts_alps.csv', 'ski_resorts_pnw.csv', 'ski_resorts_rockies.csv', 'ski_resorts_new_england.csv', 'ski_resorts_europe.csv', 'ski_resorts_na_west.csv', 'ski_resorts_na_east.csv']


# Acquire the data

For each of the ski resorts, we will get the daily forecast of temperature and precipitation as of the beginning of the season. Then we will also get the historical observed conditions, calculate snowfall, and merge them into a single dataset for later analysis.


## Daily Downscale Forecast

In contrast to the probabilistic `forecast_timeseries` function, `downscale` samples historical analogs from the forecast distribution to create ensemble timeseries.


In [ ]:
fcst_files = sk.downscale(
    loc=ski_locs,
    variables=vars,
    members=51,
    date=fcst_date,
    force=force,
)

# Because we are requesting multiple location_files, fcst_files is a
# table with multiple downscale files.  Let's combine all of them:
fcst = xr.open_mfdataset(
    fcst_files["file_name"].values,
    concat_dim="location",
    combine="nested",
)
# Align the data to the ski season:
fcst = fcst.sel(forecast_day=slice(fcst_date, fcst_end))

# We use scientific units for precip like mm day-1
# Let's make this more readable for plotting purposes:
fcst["precip"].attrs["units"] = "mm/day"

# rename "forecast_day" to "time" to match the output from data_timeseries
fcst = fcst.rename({"forecast_day": "time"})

# Remove things we don't need:
fcst = fcst.drop_vars(["temp_clim", "precip_clim", "temp_anom", "precip_anom", "analog"])


def add_season(ds, season_start_month=10, season_end_month=5):
    """Add season and season_day coordinates to a dataset based on time.

    Args:
        ds: xarray Dataset with a 'time' coordinate
        season_start_month: Month when season starts (e.g. 10 for October)
        season_end_month: Month when season ends (e.g. 5 for May)

    Returns:
        xarray Dataset with new 'season' and 'season_day' coordinates
    """
    time = ds.time
    month = time.dt.month.values
    year = time.dt.year.values

    # Calculate season_day relative to each year's season start
    season_starts = pd.to_datetime([f"{y}-{season_start_month:02d}-01" for y in year])
    time_np = time.values.astype("datetime64[D]")
    season_starts_np = season_starts.values.astype("datetime64[D]")

    # For dates before October, use previous year's October 1
    # Account for leap years in the offset
    is_leap = pd.to_datetime(season_starts_np).is_leap_year
    year_days = np.where(is_leap, 366, 365)

    days = (
        (
            time_np
            - np.where(
                month < season_start_month,
                season_starts_np - np.timedelta64(1, "D") * year_days,
                season_starts_np,
            )
        )
        .astype("timedelta64[D]")
        .astype(int)
    )
    is_summer = (month >= season_end_month) & (month < season_start_month)
    season_day = np.where(is_summer, pd.NA, days)

    # Calculate season year
    season = year
    season = np.where(month < season_start_month, season - 1, season)
    season = np.where(is_summer, pd.NA, season)

    # Add new coordinates
    ds = ds.assign_coords({"season": ("time", season), "season_day": ("time", season_day)})

    return ds


fcst = add_season(fcst)


fcst = fcst.compute()

print(fcst)

<xarray.Dataset> Size: 7MB
Dimensions:        (time: 199, ensemble: 51, location: 92)
Coordinates:
  * time           (time) datetime64[ns] 2kB 2024-10-15 ... 2025-05-01
  * location       (location) object 736B 'Hakuba' 'Rusutsu' ... 'Hunter'
    lat            (location) float64 736B 36.69 42.82 42.82 ... 42.29 42.2
    lon            (location) float64 736B 137.9 140.7 140.7 ... -74.26 -74.22
    forecast_date  datetime64[ns] 8B 2024-10-15
    season         (time) object 2kB 2024 2024 2024 2024 ... 2024 2024 2024 <NA>
    season_day     (time) object 2kB 14 15 16 17 18 19 ... 208 209 210 211 <NA>
Dimensions without coordinates: ensemble
Data variables:
    temp           (ensemble, time, location) float32 4MB 16.77 15.2 ... 9.413
    precip         (ensemble, time, location) float32 4MB 0.0 23.1 ... 2.597


## Get Historical Data

The `data_timeseries` function will load the historical daily ERA5 timeseries, which we can later compare to the `downscale` timeseries ensembles.


In [ ]:
# Get historical observed performance for each ski resort
hist_files = sk.data_timeseries(
    loc=ski_locs,
    variable=vars,
    field="vals",
    start=hist_start,
    end=min(fcst_end, today),
    frequency="daily",
    force=force,
)

# Assemble each historical file into a single xarray dataset
hist = sk.load_multihistory(hist_files)
hist = add_season(hist)

# Assign a color to each region for plotting purposes later
prefix = "ski_resorts_"
suffix = ".csv"
region = [x.replace(prefix, "").replace(suffix, "") for x in hist["location_file"].values]
regcol = [colors[reg] for reg in region]
hist = hist.assign_coords(region=("location", region), color=("location", regcol))

print(hist)

<xarray.Dataset> Size: 19MB
Dimensions:        (time: 12458, location: 92)
Coordinates:
  * time           (time) datetime64[ns] 100kB 1990-10-01 ... 2024-11-08
  * location       (location) object 736B '7 Springs' 'A-Basin' ... 'Zermatt'
    lat            (location) float64 736B 40.02 39.64 44.86 ... 39.89 46.02
    lon            (location) float64 736B -79.3 -105.9 -92.79 ... -105.8 7.752
    location_file  (location) object 736B 'ski_resorts_na_east.csv' ... 'ski_...
    season         (time) object 100kB 1990 1990 1990 1990 ... 2024 2024 2024
    season_day     (time) object 100kB 0 1 2 3 4 5 6 7 ... 32 33 34 35 36 37 38
    region         (location) <U11 4kB 'na_east' 'rockies' ... 'rockies' 'alps'
    color          (location) <U7 3kB '#777777' '#CC3311' ... '#33BBEE'
Data variables:
    temp           (time, location) float64 9MB 10.52 6.065 ... -5.545 0.9684
    precip         (time, location) float64 9MB 0.01205 0.2661 ... 6.513 0.07209


### Add history before forecast starts

We begin analyzing each ski season in October to account for snow accumulation before the mountains become skiable. If the forecast was generated after October, prepend the observed history to the forecast timeseries so that we account for weather before the forecast was generated.


In [ ]:
season_start = f"{year}-{start_month}-01"
if fcst.time[0] > np.datetime64(season_start):
    fcst = xr.concat(
        [
            hist.sel(time=slice(season_start, fcst.time[0] - pd.Timedelta(days=1)))
            .expand_dims(ensemble=fcst.ensemble)
            .transpose("ensemble", "time", "location"),
            fcst,
        ],
        dim="time",
    )
    print(fcst)

<xarray.Dataset> Size: 16MB
Dimensions:        (location: 92, ensemble: 51, time: 213)
Coordinates:
  * location       (location) object 736B '7 Springs' 'A-Basin' ... 'Zermatt'
  * ensemble       (ensemble) int64 408B 0 1 2 3 4 5 6 ... 44 45 46 47 48 49 50
  * time           (time) datetime64[ns] 2kB 2024-10-01 ... 2025-05-01
    lat            (location) float64 736B 40.02 39.64 44.86 ... 39.89 46.02
    lon            (location) float64 736B -79.3 -105.9 -92.79 ... -105.8 7.752
    location_file  (location) object 736B 'ski_resorts_na_east.csv' ... 'ski_...
    season         (time) object 2kB 2024 2024 2024 2024 ... 2024 2024 2024 <NA>
    season_day     (time) object 2kB 0 1 2 3 4 5 6 ... 207 208 209 210 211 <NA>
    region         (location) <U11 4kB 'na_east' 'rockies' ... 'rockies' 'alps'
    color          (location) <U7 3kB '#777777' '#CC3311' ... '#33BBEE'
    forecast_date  datetime64[ns] 8B 2024-10-15
Data variables:
    temp           (ensemble, time, location) float64 8M

## Calculate Snow Water Equivalent

The `calc_swe` function builds on the `snow17` model to calculate the snow water equivalent (SWE) at each location and for each ensemble. It requires that the dataset input has data values `precip` and `temp`.


In [ ]:
if "swe" not in fcst:
    fcst["swe"] = sk.hydro.calc_swe(fcst, "time")

print(fcst.data_vars)

Data variables:
    temp     (ensemble, time, location) float64 8MB 15.71 7.041 ... -6.851 1.195
    precip   (ensemble, time, location) float64 8MB 22.07 0.001423 ... 37.29
    swe      (ensemble, time, location) float64 8MB 0.0 0.0 0.0 ... 272.6 576.1


In [ ]:
if "swe" not in hist:
    hist["swe"] = sk.hydro.calc_swe(hist, "time")

print(hist.data_vars)

Data variables:
    temp     (time, location) float64 9MB 10.52 6.065 13.48 ... -5.545 0.9684
    precip   (time, location) float64 9MB 0.01205 0.2661 2.719 ... 6.513 0.07209
    swe      (time, location) float64 9MB 0.0 0.0 0.0 0.0 ... 0.0 32.8 0.03808


To calculate a seasonal average similar to the forecast's per-ensemble average we need to break the single linear `time` dimension into `season` + `season_day` dimensions.


In [ ]:
def stack_by_season(ds):
    """Convert a dataset from time dimension to season/season_day dimensions.

    Args:
        ds: xarray Dataset with time dimension and season/season_day coordinates

    Returns:
        xarray Dataset with season and season_day dimensions instead of time
    """
    valid_mask = ~ds["season"].isnull() & ~ds["season_day"].isnull()
    ds_clean = ds.isel(time=valid_mask).copy()
    ds_clean["season"] = ds_clean.season.astype(int)
    ds_clean["season_day"] = ds_clean.season_day.astype(int)
    time_df = pd.DataFrame(
        {
            "season": ds_clean.season.values,
            "season_day": ds_clean.season_day.values,
            "time": ds_clean.time.values,
        }
    )

    ds_stacked = ds_clean.set_index(time=["season", "season_day"])
    ds_final = ds_stacked.unstack("time")
    idx = pd.MultiIndex.from_product(
        [ds_final.season.values, ds_final.season_day.values], names=["season", "season_day"]
    )
    time_series = time_df.set_index(["season", "season_day"])["time"].reindex(idx)
    ds_final["time"] = xr.DataArray(
        time_series.values.reshape(len(ds_final.season), len(ds_final.season_day)),
        dims=["season", "season_day"],
    )

    return ds_final


hist = stack_by_season(hist)
# hist["swe_avg"] = hist["swe"].mean(["season", "season_day"])
print(hist.data_vars)

Data variables:
    temp     (location, season, season_day) float64 5MB 10.52 12.35 ... nan nan
    precip   (location, season, season_day) float64 5MB 0.01205 0.03876 ... nan
    swe      (location, season, season_day) float64 5MB 0.0 0.0 0.0 ... nan nan
    time     (season, season_day) datetime64[ns] 60kB 1990-10-01 ... NaT


So we can later merge, let's similarly denominate the forecast by `season_day` instead of `time`.


In [ ]:
fcst = stack_by_season(fcst).squeeze("season")
print(fcst)

<xarray.Dataset> Size: 24MB
Dimensions:        (season_day: 212, location: 92, ensemble: 51)
Coordinates:
    season         int64 8B 2024
  * season_day     (season_day) int64 2kB 0 1 2 3 4 5 ... 207 208 209 210 211
  * location       (location) object 736B '7 Springs' 'A-Basin' ... 'Zermatt'
  * ensemble       (ensemble) int64 408B 0 1 2 3 4 5 6 ... 44 45 46 47 48 49 50
    lat            (location) float64 736B 40.02 39.64 44.86 ... 39.89 46.02
    lon            (location) float64 736B -79.3 -105.9 -92.79 ... -105.8 7.752
    location_file  (location) object 736B 'ski_resorts_na_east.csv' ... 'ski_...
    region         (location) <U11 4kB 'na_east' 'rockies' ... 'rockies' 'alps'
    color          (location) <U7 3kB '#777777' '#CC3311' ... '#33BBEE'
    forecast_date  datetime64[ns] 8B 2024-10-15
Data variables:
    temp           (ensemble, location, season_day) float64 8MB 15.71 ... 3.384
    precip         (ensemble, location, season_day) float64 8MB 22.07 ... 4.798
    swe    

## Calculate Anomalies

Combine the historical and forecast datasets into a single dataset so that we can make sure they are aligned by `location`.

We don't want to highlight resorts with below-average snowfall, so let's sort the dataset and cut out the bottom half.


In [ ]:
# Let's define "climatology" as the most recent 15 years:
clim = hist.sel(season=slice(year - 16, year - 1)).mean("season", keep_attrs=True)
clim = clim.rolling(season_day=21, center=True, min_periods=1).mean()

# Let's focus our analysis on winter solstice to spring break, when most people ski
high_season = slice(81, 170)
# We're only going to plot locations with above-average snowfall
loc_avg = fcst.swe.mean(dim=["ensemble", "season_day"])
best_locations = loc_avg[loc_avg > 0.5 * loc_avg.mean()].location.values

with xr.set_options(keep_attrs=True):
    fcst_sub = fcst.sel(season_day=high_season, location=best_locations).mean("season_day")
    clim_sub = clim.sel(season_day=high_season, location=best_locations).mean("season_day")
    # Convert temperature to degK so we can analyze as ratios
    fcst_sub["temp"] = fcst_sub["temp"] + 273
    clim_sub["temp"] = clim_sub["temp"] + 273

    anom = 100 * fcst_sub / clim_sub
    anomd = fcst_sub - clim_sub

# Sort by SWE anomaly, highest first
locs_sort = anom.swe.mean(dim=["ensemble"]).sortby(anom.swe.mean(dim=["ensemble"])).location
anom = anom.sel(location=locs_sort)
anomd = anomd.sel(location=locs_sort)

[
    anom[var].attrs.update(
        {"units": "%", "long_name": f'{anom[var].attrs.get("long_name", var)} anomaly'}
    )
    for var in anom.data_vars
];

# Results


In [ ]:
def plot_boxes(
    fcst: xr.DataArray,
    title: str = "",
    vline=None,
    legend_loc: str = "center right",
    ax=None,
):
    """Plot predictions and observed values as a box-and-whisker plot."""
    # extract a table of seasonal averages (across all days) per location
    box_data = fcst.to_dataframe(dim_order=["location", "ensemble"])
    box_data = box_data[fcst.name].unstack(level=0).to_numpy()

    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 10))  # Create a new figure if no axis is provided

    plt_box = ax.boxplot(
        box_data,
        showfliers=False,
        vert=False,
        labels=fcst.location.values,
        patch_artist=True,
        meanline=True,
        showmeans=True,
        meanprops=dict(linewidth=1, color="white"),
        medianprops=dict(linewidth=0, color="gray", alpha=0),
        whiskerprops=dict(linewidth=0.7, color="gray"),
        capprops=dict(linewidth=0.7, color="gray"),
        boxprops=dict(linewidth=0.7, color="gray"),
    )

    [patch.set_facecolor(color) for patch, color in zip(plt_box["boxes"], fcst.color.values)]
    ax.set_xlabel(f"{fcst.long_name} ({fcst.units})")
    ax.set_title(title)
    legend_names = ["japan", "alps", "pnw", "rockies", "new_england"]
    legend_handles = plt_box["boxes"][: len(legend_names)]

    if vline is not None:
        ax.axvline(vline, color="grey", linestyle="--", linewidth=1)

    if legend_loc == "none":
        ax.set_yticklabels([])
    else:
        leg = ax.legend(legend_handles, legend_names, loc=legend_loc)
        for patch, reg in zip(leg.get_patches(), legend_names):
            patch.set_facecolor(colors[reg])


fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 15))
plot_boxes(anom["swe"], vline=100, legend_loc="upper left", ax=ax1)
plot_boxes(anom["precip"], vline=100, legend_loc="none", ax=ax2)
plot_boxes(anomd["temp"], vline=0, legend_loc="none", ax=ax3)
plt.tight_layout()

#### Ensemble Variation at 4 Resorts


In [ ]:
focus_names = ["Whistler", "Kitzbuehl", "Copper", "Sugarloaf"]
focus_names = ["RED", "Alpental", "Revelstoke", "Crystal Mtn"]


def plot_ensembles(fcst, clim, var="temp", title=False):
    """Show ensemble values for a given variable."""
    x_val = "season_day"

    favg = fcst.mean(dim="ensemble", keep_attrs=True)
    clr = fcst["color"].values.tolist()

    fcst[var].plot.line(x=x_val, color="grey", alpha=0.1, add_legend=False)
    favg[var].plot.line(x=x_val, color="black", linewidth=2, add_legend=False)
    clim[var].plot.line(x=x_val, color=clr, alpha=0.8, linewidth=2, add_legend=False)

    plt.title(fcst["location"].values if title else "")
    plt.xlabel("")


(fig, axs) = plt.subplots(
    nrows=3,
    ncols=len(focus_names),
    sharex=True,
    sharey="row",
    figsize=(5 * len(focus_names), 15),
)

for idx in range(len(focus_names)):
    fcst_loc = fcst.sel(location=focus_names[idx])
    clim_loc = clim.sel(location=focus_names[idx])
    plt.sca(axs[0, idx])
    plot_ensembles(fcst_loc, clim_loc, "swe", title=True)
    plt.sca(axs[1, idx])
    plot_ensembles(fcst_loc, clim_loc, "temp")
    plt.axhline(0, color="k", linestyle="--")
    plt.sca(axs[2, idx])
    plot_ensembles(fcst_loc, clim_loc, "precip")
    plt.gca().set_ylim((0, 40))